# Генерация мелодий

## Создание функции для генерации мелодий

**Функция, которая выводит массив со ступенями нот, используемых в цепочках аккордов**

In [11]:
def chord_step(start):
    values = []
    current_value = start
    for _ in range(3):
        values.append(current_value)
        current_value += 2
        if current_value > 7:
            current_value = current_value % 7  
            if current_value == 0:
                current_value = 7
    return values

def chord_st_chain(input_list):
    results = {}
    for i, value in enumerate(input_list):
        results[f"ch_{i+1}"] = chord_step(value)
    return results
    

In [12]:
result = chord_st_chain([1, 7, 6, 5])
result

{'ch_1': [1, 3, 5], 'ch_2': [7, 2, 4], 'ch_3': [6, 1, 3], 'ch_4': [5, 7, 2]}

**Функция, которая выводит ритмический рисунок для одного аккорда**

In [18]:
import random

def rhythmic_pattern():
    values = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
    weights = [8, 7, 6, 5, 4, 3, 2, 1]  
    pattern = []
    current_sum = 0
    
    while current_sum < 1:
        value = random.choices(values, weights)[0]
        if current_sum + value <= 1:
            pattern.append(value)
            current_sum += value
    random.shuffle(pattern)
    return pattern

In [19]:
rhythmic_pattern()

[0.125, 0.375, 0.375, 0.125]

In [24]:
def generate_melody(chord_st_chain):
    all_steps = []
    all_durations = []
    
    for notes in chord_st_chain.values():
        rythm = rhythmic_pattern()
        melody_notes = [random.choice(notes) for _ in rythm]
        all_steps.extend(melody_notes)
        all_durations.extend(rythm)
    
    return [all_steps, all_durations]

generate_melody(chord_st_chain([1, 7, 6, 5]))


[[5, 1, 5, 3, 4, 2, 1, 1, 3, 7, 7, 2],
 [0.5, 0.125, 0.25, 0.125, 0.125, 0.875, 0.375, 0.125, 0.5, 0.375, 0.125, 0.5]]

In [20]:
import math

def note_frequency(base_freq, step):
    a = 2 ** (1 / 12)
    return base_freq * (a ** step)

def melody_freq(melody, tone=440.0):
    minor_intervals = [0, 2, 3, 5, 7, 8, 10]
    steps = melody[0]
    durations = melody[1]

    frequencies = [note_frequency(tone, minor_intervals[step - 1]) for step in steps]
    return [frequencies, durations]

# Пример использования
melody = [[3, 3, 1, 1, 4, 4, 2, 6, 1, 3, 7, 7, 7],
          [0.125, 0.125, 0.125, 0.625, 0.375, 0.5, 0.125, 0.375, 0.5, 0.125, 0.125, 0.5, 0.375]]

melody_freq = melody_freq(melody)


In [17]:
import numpy as np
from scipy.io.wavfile import write
import IPython.display as ipd

def sine_wave(frequency, duration, sample_rate=44100, amplitude=0.5):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return wave

def melody_play(melody_freq, duration=10.0, sample_rate=44100):
    frequencies = melody_freq[0]
    durations = melody_freq[1]
    total_wave = np.array([])

    for freq, dur in zip(frequencies, durations):
        wave_duration = dur * (duration / 4)
        wave = sine_wave(freq, wave_duration, sample_rate)
        total_wave = np.concatenate((total_wave, wave))
    
    audio_wave = np.int16(total_wave * 32767)
    write('melody.wav', sample_rate, audio_wave)
    return ipd.Audio('melody.wav')

melody_play(melody_freq)